<h1 align='center'>Final Capstone: <i>Betcha Can't Guess What I Watched</i></h1>
<h2 align='center'>Philip Bowman</h2>
<h1 align='center'>Final Product</h1>

This is the final selected model and its implementation for demo purposes.

***This product uses the TMDb API but is not endorsed or certified by TMDb.***

<img src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRw%0D%0AOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB2aWV3Qm94PSIwIDAgMTg1LjA0IDEzMy40Ij48ZGVm%0D%0Acz48c3R5bGU+LmNscy0xe2ZpbGw6dXJsKCNsaW5lYXItZ3JhZGllbnQpO308L3N0eWxlPjxsaW5l%0D%0AYXJHcmFkaWVudCBpZD0ibGluZWFyLWdyYWRpZW50IiB5MT0iNjYuNyIgeDI9IjE4NS4wNCIgeTI9%0D%0AIjY2LjciIGdyYWRpZW50VW5pdHM9InVzZXJTcGFjZU9uVXNlIj48c3RvcCBvZmZzZXQ9IjAiIHN0%0D%0Ab3AtY29sb3I9IiM5MGNlYTEiLz48c3RvcCBvZmZzZXQ9IjAuNTYiIHN0b3AtY29sb3I9IiMzY2Jl%0D%0AYzkiLz48c3RvcCBvZmZzZXQ9IjEiIHN0b3AtY29sb3I9IiMwMGIzZTUiLz48L2xpbmVhckdyYWRp%0D%0AZW50PjwvZGVmcz48dGl0bGU+QXNzZXQgNDwvdGl0bGU+PGcgaWQ9IkxheWVyXzIiIGRhdGEtbmFt%0D%0AZT0iTGF5ZXIgMiI+PGcgaWQ9IkxheWVyXzEtMiIgZGF0YS1uYW1lPSJMYXllciAxIj48cGF0aCBj%0D%0AbGFzcz0iY2xzLTEiIGQ9Ik01MS4wNiw2Ni43aDBBMTcuNjcsMTcuNjcsMCwwLDEsNjguNzMsNDlo%0D%0ALS4xQTE3LjY3LDE3LjY3LDAsMCwxLDg2LjMsNjYuN2gwQTE3LjY3LDE3LjY3LDAsMCwxLDY4LjYz%0D%0ALDg0LjM3aC4xQTE3LjY3LDE3LjY3LDAsMCwxLDUxLjA2LDY2LjdabTgyLjY3LTMxLjMzaDMyLjlB%0D%0AMTcuNjcsMTcuNjcsMCwwLDAsMTg0LjMsMTcuN2gwQTE3LjY3LDE3LjY3LDAsMCwwLDE2Ni42Myww%0D%0AaC0zMi45QTE3LjY3LDE3LjY3LDAsMCwwLDExNi4wNiwxNy43aDBBMTcuNjcsMTcuNjcsMCwwLDAs%0D%0AMTMzLjczLDM1LjM3Wm0tMTEzLDk4aDYzLjlBMTcuNjcsMTcuNjcsMCwwLDAsMTAyLjMsMTE1Ljdo%0D%0AMEExNy42NywxNy42NywwLDAsMCw4NC42Myw5OEgyMC43M0ExNy42NywxNy42NywwLDAsMCwzLjA2%0D%0ALDExNS43aDBBMTcuNjcsMTcuNjcsMCwwLDAsMjAuNzMsMTMzLjM3Wm04My45Mi00OWg2LjI1TDEy%0D%0ANS41LDQ5aC04LjM1bC04LjksMjMuMmgtLjFMOTkuNCw0OUg5MC41Wm0zMi40NSwwaDcuOFY0OWgt%0D%0ANy44Wm0yMi4yLDBoMjQuOTVWNzcuMkgxNjcuMVY3MGgxNS4zNVY2Mi44SDE2Ny4xVjU2LjJoMTYu%0D%0AMjVWNDloLTI0Wk0xMC4xLDM1LjRoNy44VjYuOUgyOFYwSDBWNi45SDEwLjFaTTM5LDM1LjRoNy44%0D%0AVjIwLjFINjEuOVYzNS40aDcuOFYwSDYxLjlWMTMuMkg0Ni43NVYwSDM5Wm00MS4yNSwwaDI1VjI4%0D%0ALjJIODhWMjFoMTUuMzVWMTMuOEg4OFY3LjJoMTYuMjVWMGgtMjRabS03OSw0OUg5VjU3LjI1aC4x%0D%0AbDksMjcuMTVIMjRsOS4zLTI3LjE1aC4xVjg0LjRoNy44VjQ5SDI5LjQ1bC04LjIsMjMuMWgtLjFM%0D%0AMTMsNDlIMS4yWm0xMTIuMDksNDlIMTI2YTI0LjU5LDI0LjU5LDAsMCwwLDcuNTYtMS4xNSwxOS41%0D%0AMiwxOS41MiwwLDAsMCw2LjM1LTMuMzcsMTYuMzcsMTYuMzcsMCwwLDAsNC4zNy01LjVBMTYuOTEs%0D%0AMTYuOTEsMCwwLDAsMTQ2LDExNS44YTE4LjUsMTguNSwwLDAsMC0xLjY4LTguMjUsMTUuMSwxNS4x%0D%0ALDAsMCwwLTQuNTItNS41M0ExOC41NSwxOC41NSwwLDAsMCwxMzMuMDcsOTksMzMuNTQsMzMuNTQs%0D%0AMCwwLDAsMTI1LDk4SDExMy4yOVptNy44MS0yOC4yaDQuNmExNy40MywxNy40MywwLDAsMSw0LjY3%0D%0ALjYyLDExLjY4LDExLjY4LDAsMCwxLDMuODgsMS44OCw5LDksMCwwLDEsMi42MiwzLjE4LDkuODcs%0D%0AOS44NywwLDAsMSwxLDQuNTIsMTEuOTIsMTEuOTIsMCwwLDEtMSw1LjA4LDguNjksOC42OSwwLDAs%0D%0AMS0yLjY3LDMuMzQsMTAuODcsMTAuODcsMCwwLDEtNCwxLjgzLDIxLjU3LDIxLjU3LDAsMCwxLTUs%0D%0ALjU1SDEyMS4xWm0zNi4xNCwyOC4yaDE0LjVhMjMuMTEsMjMuMTEsMCwwLDAsNC43My0uNSwxMy4z%0D%0AOCwxMy4zOCwwLDAsMCw0LjI3LTEuNjUsOS40Miw5LjQyLDAsMCwwLDMuMS0zLDguNTIsOC41Miww%0D%0ALDAsMCwxLjItNC42OCw5LjE2LDkuMTYsMCwwLDAtLjU1LTMuMiw3Ljc5LDcuNzksMCwwLDAtMS41%0D%0ANy0yLjYyLDguMzgsOC4zOCwwLDAsMC0yLjQ1LTEuODUsMTAsMTAsMCwwLDAtMy4xOC0xdi0uMWE5%0D%0ALjI4LDkuMjgsMCwwLDAsNC40My0yLjgyLDcuNDIsNy40MiwwLDAsMCwxLjY3LTUsOC4zNCw4LjM0%0D%0ALDAsMCwwLTEuMTUtNC42NSw3Ljg4LDcuODgsMCwwLDAtMy0yLjczLDEyLjksMTIuOSwwLDAsMC00%0D%0ALjE3LTEuMywzNC40MiwzNC40MiwwLDAsMC00LjYzLS4zMmgtMTMuMlptNy44LTI4LjhoNS4zYTEw%0D%0ALjc5LDEwLjc5LDAsMCwxLDEuODUuMTcsNS43Nyw1Ljc3LDAsMCwxLDEuNy41OCwzLjMzLDMuMzMs%0D%0AMCwwLDEsMS4yMywxLjEzLDMuMjIsMy4yMiwwLDAsMSwuNDcsMS44MiwzLjYzLDMuNjMsMCwwLDEt%0D%0ALjQyLDEuOCwzLjM0LDMuMzQsMCwwLDEtMS4xMywxLjIsNC43OCw0Ljc4LDAsMCwxLTEuNTcuNjUs%0D%0AOC4xNiw4LjE2LDAsMCwxLTEuNzguMkgxNjVabTAsMTQuMTVoNS45YTE1LjEyLDE1LjEyLDAsMCwx%0D%0ALDIuMDUuMTUsNy44Myw3LjgzLDAsMCwxLDIsLjU1LDQsNCwwLDAsMSwxLjU4LDEuMTcsMy4xMywz%0D%0ALjEzLDAsMCwxLC42MiwyLDMuNzEsMy43MSwwLDAsMS0uNDcsMS45NSw0LDQsMCwwLDEtMS4yMywx%0D%0ALjMsNC43OCw0Ljc4LDAsMCwxLTEuNjcuNyw4LjkxLDguOTEsMCwwLDEtMS44My4yaC03WiIvPjwv%0D%0AZz48L2c+PC9zdmc+"
width="100" height="50" align='left'>

In [10]:
import os
from os.path import join
import pandas as pd
import numpy as np
import time
from IPython.display import clear_output, display, Markdown
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tmdbv3api import Movie, TMDb

In [2]:
movies_dir = r'C:\Users\philb\Datasets\movies_post_exploration'
movies_file = 'movies.pkl'
unpop_file = 'less_popular_movies.pkl'
pool_file = 'recommendation_pool.pkl' #won't need this until later

In [3]:
#this cell takes all the textual features in the movies dataframe and puts it altogether into the movies_strings variable 

movies = pd.read_pickle(join(movies_dir, movies_file))
unpop_movies_ids = pd.read_pickle(join(movies_dir, unpop_file)).index

text_features = ['spoken_languages', 'genres', 'overview', 'tagline', 'keywords', 'production_companies', 'acting_top_5', 'director', 'writers']
list_features = ['spoken_languages', 'genres', 'keywords', 'production_companies', 'acting_top_5', 'director', 'writers']

movies_features = movies[text_features].copy()
movies_features = movies_features.fillna(' ')

def unpack_list(x):
    string_rep = ' '
    for item in x:
        string_rep = string_rep + str(item) + ' '
    return string_rep

for column in list_features:
    movies_features[column] = movies_features[column].apply(unpack_list)
    
add_space_columns = [column for column in movies_features.columns if column not in list_features]

for column in add_space_columns:
    movies_features[column] = movies_features[column].apply(lambda x: x + ' ')
    
movies_text = movies_features.sum(axis=1)
movies_titles = movies.title
pop_movies_indicies = [index for index in movies_titles.index if index not in unpop_movies_ids]
pop_movies = movies_titles[pop_movies_indicies]

del movies, text_features, list_features, movies_features, add_space_columns, unpop_movies_ids

rec_pool = pd.read_pickle(join(movies_dir, pool_file))
rec_idxs = [movies_titles.index.get_loc(movie_id) for movie_id in rec_pool.index]

In [4]:
def cosine_pipe(text_docs, feature_space):
    cv = CountVectorizer(max_features=feature_space, stop_words='english')
    feature_csr = cv.fit_transform(text_docs)
    all_vars = (cv, feature_csr)
    return all_vars

tokenizer, movie_vectors = cosine_pipe(movies_text, None)
rec_vectors = movie_vectors[rec_idxs]

In [5]:
API_KEY_PATH = r'C:\Users\philb\Datasets\API_KEY.txt'
with open(API_KEY_PATH) as f:
    API_KEY = f.readline()
f.close()

In [6]:
tmdb = TMDb()
movie = Movie()
tmdb.api_key = API_KEY

In [19]:
def get_id(search_term):
    print('------------')
    print('MOVIE SEARCH')
    print('------------')
    try:
        page1 = [(i, movie) for i, movie in enumerate(movie.search(search_term))]
        for tup in page1:
            print(tup[0], tup[1])
        time.sleep(.01)
        movie_sel = int(input('Enter index # of selected movie: '))
        movie_id = page1[movie_sel][1].id
        clear_output(wait=True)
        return movie_id
    except:
        print('No movies were found using that query.')

def top_X_obscure(movie_id, x_sim=10):
    cos_sim = cosine_similarity(rec_vectors, movie_vectors[movies_titles.index.get_loc(movie_id)])
    if movie_id in rec_idxs:
        top_X_idx = pd.DataFrame(cos_sim, index=rec_pool.index).loc[:, 0].sort_values(ascending=False).iloc[1:(x_sim+1)].index
    elif movie_id not in rec_idxs:
        top_X_idx = pd.DataFrame(cos_sim, index=rec_pool.index).loc[:, 0].sort_values(ascending=False).iloc[0:(x_sim)].index
    print(f'Top {x_sim} Obscure Movies Similar to {movies_titles.loc[movie_id]}')
    movie_links = ['https://www.themoviedb.org/movie/'+str(i) for i in top_X_idx]
    movie_df = pd.DataFrame(movies_titles.loc[top_X_idx])
    movie_df['links'] = movie_links
    return display(Markdown(movie_df.to_markdown()))
        
        
def user_query(user_input, x_sim=10):
    vector = tokenizer.transform([user_input])
    cos_sim = cosine_similarity(rec_vectors, vector)
    top_X_idx = pd.DataFrame(cos_sim, index=rec_pool.index).loc[:, 0].sort_values(ascending=False).iloc[0:(x_sim)].index
    print(f'Top {x_sim} Obscure Movies Similar to user search: "{user_input}"')
    movie_links = ['https://www.themoviedb.org/movie/'+str(i) for i in top_X_idx]
    movie_df = pd.DataFrame(movies_titles.loc[top_X_idx])
    movie_df['links'] = movie_links
    return display(Markdown(movie_df.to_markdown()))

def get_recommendations(query, kind='movie', x_sim=10):
    if kind == 'movie':
        top_X_obscure(get_id(query), x_sim)
    elif kind == 'query':
        user_query(query, x_sim)

In [21]:
get_recommendations('lord of the rings', kind='movie')

Top 10 Obscure Movies Similar to The Lord of the Rings: The Two Towers


|     id | title                                   | links                                   |
|-------:|:----------------------------------------|:----------------------------------------|
|   7234 | Wizards of the Lost Kingdom             | https://www.themoviedb.org/movie/7234   |
|   9070 | Mighty Morphin Power Rangers: The Movie | https://www.themoviedb.org/movie/9070   |
|   9964 | Bad Taste                               | https://www.themoviedb.org/movie/9964   |
|   9826 | The Phantom                             | https://www.themoviedb.org/movie/9826   |
|   2274 | The Seeker: The Dark Is Rising          | https://www.themoviedb.org/movie/2274   |
| 275696 | 3 Nights in the Desert                  | https://www.themoviedb.org/movie/275696 |
|  44571 | The King of Fighters                    | https://www.themoviedb.org/movie/44571  |
|  24115 | How to Eat Fried Worms                  | https://www.themoviedb.org/movie/24115  |
|   8216 | Meet the Feebles                        | https://www.themoviedb.org/movie/8216   |
|   1717 | All the King's Men                      | https://www.themoviedb.org/movie/1717   |